<h1> 3. Priprema podataka </h1>

Cilj je u ovoj bilježnici transformirati skup za testiranje (definirati nove značajke) jednako kao skup za učenje. 
Na kraju ćemo spojiti oba skupa s makroekonomskim indikatorima. Makroekonomski podaci su dani po godinama.

Output ove datoteke su datoteke spremne za treniranje modela (enkodirane i normalizirane). Prije treniranja se radi još selekcija značajki u bilježnici "feature_engineering".

Izvori makroekonomskih podataka:

https://www.rba.hr/mala-poduzeca-i-obrtnici/istrazivanja-i-analize/makroekonomski-pokazatelji (2018.)

https://www.hnb.hr/documents/20182/121813/h-ekonomski_indikatori.xlsx/c29372bf-0fc0-e472-41e4-6853925244f8?t=1555578653745 (2000.-2018.)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
### prvo čitanje skupa za testiranje
mydateparser = lambda c: pd.to_datetime(c, format='%d.%m.%Y', errors='coerce')
test = pd.read_excel('eval_dataset_nan.xlsx')
test['DATUM_OTVARANJA'] = pd.to_datetime(test['DATUM_OTVARANJA'], format='%d.%m.%Y')

In [ ]:
### spremanje u pogodniji format
test.to_hdf('test_dataset.h5', key = 'test')

In [3]:
### ako je datoteka 'test_dataset.h5' sadržana u direktoriju, ovako čitati
test = pd.read_hdf('test_dataset.h5', key = 'test')

In [4]:
test.head()

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,0,1049000,5585453,2012-07-14,2013-07-15,273555.01,2,1410,TM0109,L,3.60,A,64,NaN
1,1,481302,752335,2016-01-24,2016-01-24,50970.78,2,1410,TM0109,L,0.20,A,56,NaN
2,2,838192,6623548,2018-12-26,2018-12-26,8969.36,2,1410,TM0109,L,0.02,A,52,NaN
3,3,174575,425060,2012-08-04,2012-08-04,16142.48,2,1410,TM0109,L,2.30,C,76,NaN
4,4,14884,1183149,2011-03-27,2012-03-28,45315.31,2,1410,TM0109,L,3.30,C,53,NaN


<h3> Istraživanje testnog skupa </h3>

U ovom dijelu želimo vidjeti ima li podataka s "nelogičnostima" na koje smo i naišli i u skupu za treniranje te da li se i ovdje nalaze produljivani ugovori (što nam utječe na odabir značajki).

In [5]:
test_nelogicni = test[test['DATUM_OTVARANJA']>= test['PLANIRANI_DATUM_ZATVARANJA']]

In [6]:
test_nelogicni

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
1,1,481302,752335,2016-01-24,2016-01-24,50970.78,2,1410,TM0109,L,0.20,A,56,NaN
2,2,838192,6623548,2018-12-26,2018-12-26,8969.36,2,1410,TM0109,L,0.02,A,52,NaN
3,3,174575,425060,2012-08-04,2012-08-04,16142.48,2,1410,TM0109,L,2.30,C,76,NaN
9,9,888283,9088014,2018-11-23,2018-11-23,54123.73,1,1410,TM0109,L,0.20,A,38,NaN
14,14,385859,1211979,2018-12-05,2018-12-05,25506.30,2,1410,TM0109,L,0.02,A,61,NaN
15,15,112284,11405495,2016-04-19,2016-04-19,53692.27,2,1410,TM0109,L,0.10,A,73,NaN
21,21,496265,10039665,2018-12-15,2018-10-13,10223.43,1,1410,TM0109,L,0.05,A,56,NaN
29,29,1221746,5995009,2013-04-21,2013-04-21,30000.00,1,1410,TM0109,L,2.70,A,85,NaN
35,35,358636,11391432,2018-07-03,2018-07-03,76833.44,2,1410,TM0109,L,0.02,A,55,NaN
38,38,459038,11278595,2018-10-22,2018-10-22,77972.99,2,1410,TM0109,L,0.05,A,65,NaN


Pitamo se postoje li i ovdje ugovori s ugovorenim iznosom 0.

In [7]:
test_iznos_nula =  test[test['UGOVORENI_IZNOS'] == 0]

In [8]:
test_iznos_nula.shape

(42, 14)

Pitamo se postoje li produljivani ugovori.

In [9]:
test_duplicates = test.drop_duplicates(subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE']) ## ugovori (bez datuma otvaranja)
print(test.shape)
print(test_duplicates.shape)

(84903, 14)
(81559, 14)


<h3> Transformacije </h3>

U ovom dijelu radimo iste transformacije na skupu za testiranje koje smo radili i na skupu za treniranje u prethodnoj bilježnici ("data_processing").

In [10]:
test['GODINA_OTVARANJA'] = test['DATUM_OTVARANJA'].dt.year
test['PLANIRANA_DULJINA_UGOVORA'] = (test['PLANIRANI_DATUM_ZATVARANJA']-test['DATUM_OTVARANJA']).dt.days

In [11]:
grouped = test.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])

def produljenja(series):
    n = series.count()
    return range(0,n)

test['PRODULJENJA'] = grouped['DATUM_OTVARANJA'].transform(produljenja)

In [12]:
test.groupby(['PRODULJENJA']).size()

PRODULJENJA
0    81559
1     2456
2      624
3      193
4       56
5       14
6        1
dtype: int64

In [13]:
del_col_list = ['instance_id', 'DATUM_OTVARANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'OZNAKA_PARTIJE']
test.drop(del_col_list, axis = 1, inplace = True)

In [14]:
test.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'VISINA_KAMATE', 'TIP_KAMATE', 'STAROST',
       'PRIJEVREMENI_RASKID', 'GODINA_OTVARANJA', 'PLANIRANA_DULJINA_UGOVORA',
       'PRODULJENJA'],
      dtype='object')

In [15]:
test.shape

(84903, 13)

In [16]:
test.to_hdf('test_nakon_transformacija.h5', key = 'test')

<h3> Učitavanje skupa za učenje </h3>

In [17]:
# output datoteke data_preprocessing
train = pd.read_hdf('train_dataset.h5', key ='df') 

In [18]:
train.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'VISINA_KAMATE',
       'TIP_KAMATE', 'STAROST', 'PRIJEVREMENI_RASKID', 'PRODULJENJA',
       'GODINA_OTVARANJA', 'PLANIRANA_DULJINA_UGOVORA'],
      dtype='object')

In [19]:
del train['OZNAKA_PARTIJE']

U sljedećoj ćeliji promatramo postoji li preklapanje skupova klijenata jer nam to utječe na odabir značajke 'KLIJENT_ID'.

In [20]:
train_klijenti = train['KLIJENT_ID'].unique()
test_klijenti = test['KLIJENT_ID'].unique()
presjek = set(set(test_klijenti).intersection(set(train_klijenti)))

In [21]:
train_klijenti.size

220284

In [22]:
print('Veličina presjeka: ',len(presjek))
print('Postotak presjeka u skupu za učenje: ', '{:2.2f}'.format( len(presjek)/ len(set(train_klijenti))*100), '%')
print('Postotak presjeka u skupu za testiranje: ', '{:2.2f}'.format( len(presjek)/ len(set(test_klijenti))*100), '%')

Veličina presjeka:  38949
Postotak presjeka u skupu za učenje:  17.68 %
Postotak presjeka u skupu za testiranje:  59.01 %


<h3> Makroekonomski indikatori </h3>

Učitavamo podatke iz tablice. Tablicu smo dobili sa stranice HNB (glavni makroekonomski indikatori) te smo je nadopunili podacima s makroekonomskim podacima iz RBA arhive.

In [26]:
# Čitam tablicu ekonomskih indikatora te je pretvaramo u dataframe oblik
econ = pd.read_excel('./data/external/ekonomski_indikatori.xlsx')

econ.dropna(thresh=int(econ.shape[1] * .9), axis = 0, inplace = True)
column_names = econ.columns
# Drop unwanted columns
econ.drop(column_names[0], inplace=True, axis=1)

econ.rename(columns={'Unnamed: 2': '2000', 'Unnamed: 3': '2001', 
                   'Unnamed: 4': '2002', 'Unnamed: 5': '2003', 
                   'Unnamed: 6': '2004', 'Unnamed: 7': '2005',
                  'Unnamed: 8': '2006', 'Unnamed: 9': '2007', 
                  'Unnamed: 10': '2008', 'Unnamed: 11': '2009',
                  'Unnamed: 12': '2010', 'Unnamed: 13': '2011',
                  'Unnamed: 14': '2012', 'Unnamed: 15': '2013',
                  'Unnamed: 16': '2014', 'Unnamed: 17': '2015',
                  'Unnamed: 18': '2016', 'Unnamed: 19': '2017',
                  'Unnamed: 20': '2018'}, inplace=True)

econ.drop([4], axis = 0, inplace = True)

econ2 = econ.transpose(copy = True)
new_column_names = econ2.iloc[0,:]
econ2.columns = new_column_names

econ2.reset_index()

Unnamed: 1,index,Area (square km),Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),...,"Gross international reserves (in terms of months of imports of goods and services, end of year)",Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years of age) f","Employment rate (ILO, persons above 15 years of age) f"
0,Unnamed: 1,Area (square km),Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, const...",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),...,Gross international reserves (in terms of mont...,Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+...,Consolidated general government net lending (+...,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years...","Employment rate (ILO, persons above 15 years o..."
1,2000,56594,4.426,180241,23610.7,5334.56,3.7675,4.6,-516.727,-2.18852,...,4.85331,7.59833,8.15534,7.63385,8.28737,....,....,35.4986,16.05,42.6
2,2001,56594,4.30549,194271,26003.3,6039.56,3.45241,3.8,-758.857,-2.91831,...,5.81721,7.37003,8.35604,7.47101,8.33915,....,....,36.4755,15.8,41.8
3,2002,56594,4.30538,211500,28554.2,6632.21,5.25336,1.7,-2016.88,-7.06332,...,5.20306,7.44229,7.14574,7.40698,7.87249,-7365.19,-3.48235,36.6133,14.8,43.3
4,2003,56594,4.30572,232444,30729.3,7136.85,5.58119,1.8,-1811.73,-5.89579,...,5.52068,7.64691,6.11851,7.56425,6.70445,-10863.8,-4.67371,38.1037,14.25,43.1
5,2004,56594,4.31086,250565,33427.9,7754.34,3.91456,2.1,-1355.35,-4.05455,...,5.0642,7.67123,5.63688,7.49568,6.03122,-13038.5,-5.20365,40.2784,13.8,43.5
6,2005,56594,4.31249,269779,36456.5,8453.7,4.10956,3.3,-1892.06,-5.18991,...,5.37051,7.37563,6.23363,7.40005,5.94996,-10413.5,-3.86001,41.1652,12.7,43.3
7,2006,56594,4.31353,294150,40168.8,9312.29,4.86782,3.2,-2612.7,-6.5043,...,5.60497,7.34508,5.5784,7.32285,5.83917,-9971.92,-3.39008,38.652,11.2,43.6
8,2007,56594,4.31197,322464,43956.2,10194,5.28365,2.9,-3138.26,-7.13952,...,5.48831,7.32513,4.98546,7.33602,5.36599,-7880.62,-2.44388,37.2593,9.9,47.6
9,2008,56594,4.3098,347750,48143.6,11170.7,2.03646,6.1,-4226.91,-8.7798,...,4.88824,7.32442,5.1555,7.22318,4.93442,-9604.64,-2.76194,39.0003,8.5,48.6


In [27]:
indikatori = econ2.copy()
indikatori = indikatori.iloc[1:, :]
indikatori['GODINE']=range(2000,2019)
drop_col = [0,1,2, 3,7,11, 13, 14, 15, 16, 17, 18, 24]
indikatori.columns[drop_col].values
indikatori.drop(indikatori.columns[drop_col].values, axis = 1, inplace = True)

In [28]:
indikatori

Unnamed: 1,GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (as of % GDP),Exports of goods and services (as of % GDP),Imports of goods and services (as of % GDP),External debt (as of % GDP),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Employment rate (ILO, persons above 15 years of age) f",GODINE
2000,5334.56,3.7675,4.6,-2.18852,35.215,39.6182,51.9406,7.63385,8.28737,....,....,35.4986,42.6,2000
2001,6039.56,3.45241,3.8,-2.91831,37.2483,42.3117,52.3366,7.47101,8.33915,....,....,36.4755,41.8,2001
2002,6632.21,5.25336,1.7,-7.06332,36.24,45.6459,53.036,7.40698,7.87249,-7365.19,-3.48235,36.6133,43.3,2002
2003,7136.85,5.58119,1.8,-5.89579,39.7021,46.3605,64.7067,7.56425,6.70445,-10863.8,-4.67371,38.1037,43.1,2003
2004,7754.34,3.91456,2.1,-4.05455,39.7148,45.6236,68.6055,7.49568,6.03122,-13038.5,-5.20365,40.2784,43.5,2004
2005,8453.7,4.10956,3.3,-5.18991,39.4753,45.5899,71.2905,7.40005,5.94996,-10413.5,-3.86001,41.1652,43.3,2005
2006,9312.29,4.86782,3.2,-6.5043,39.771,46.505,74.0001,7.32285,5.83917,-9971.92,-3.39008,38.652,43.6,2006
2007,10194,5.28365,2.9,-7.13952,39.0262,46.2966,76.7145,7.33602,5.36599,-7880.62,-2.44388,37.2593,47.6,2007
2008,11170.7,2.03646,6.1,-8.7798,38.4784,46.5081,84.3103,7.22318,4.93442,-9604.64,-2.76194,39.0003,48.6,2008
2009,10492.6,-7.29143,2.4,-5.09269,34.5037,38.1936,101.002,7.33955,5.28037,-19844,-5.98853,48.2978,48.2,2009


<h4> Data fusion </h4>

Napravit ćemo left join na training i test skupu s tablicom makroekonomskih podataka po varijablama 'GODINA_OTVARANJA' i 'GODINE'.

In [29]:
# TRAINING SET

train_merged = pd.merge(train, indikatori, left_on=['GODINA_OTVARANJA'], right_on = ['GODINE'], how = 'left')

In [30]:
train_merged.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'VISINA_KAMATE', 'TIP_KAMATE', 'STAROST',
       'PRIJEVREMENI_RASKID', 'PRODULJENJA', 'GODINA_OTVARANJA',
       'PLANIRANA_DULJINA_UGOVORA', 'GDP per capita (in EUR)  ',
       'GDP - year-on-year rate of growth (in %, constant prices) ',
       'Average CPI year-on-year inflation rate ',
       'Current account balance (as of % GDP) ',
       'Exports of goods and services (as of % GDP) ',
       'Imports of goods and services (as of % GDP) ',
       'External debt (as of % GDP) ', 'Average exchange rate (HRK : 1 EUR) ',
       'Average exchange rate (HRK : 1 USD) ',
       'Consolidated general government net lending (+)/borrowing (-) (million HRK) e',
       'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e',
       'General government debt (as % of GDP) e',
       'Employment rate (ILO, persons above 15 years of age) f', 'GODINE'],
      dtype='ob

Preimenovat ćemo stupce koji se odnose na makroekonomske indikatore kako bi bilo lakše s njima baratati.

In [31]:
train_merged.rename(columns={'GDP per capita (in EUR)  ': 'ind1-poc', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-poc', 
                   'Average CPI year-on-year inflation rate ': 'ind3-poc', 'Current account balance (as of % GDP) ': 'ind4-poc', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-poc', 'Imports of goods and services (as of % GDP) ': 'ind6-poc',
                  'External debt (as of % GDP) ': 'ind7-poc', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-poc', 'Average exchange rate (HRK : 1 USD) ': 'ind9-poc',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-poc', 'General government debt (as % of GDP) e': 'ind11-poc',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-poc'}, inplace=True)

In [32]:
del train_merged['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']
del train_merged['GODINE']

In [33]:
# TEST DATASET
test_merged = pd.merge(test,indikatori,left_on=['GODINA_OTVARANJA'], right_on=['GODINE'], how='left')
test_merged.rename(columns={'GDP per capita (in EUR)  ': 'ind1-poc', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-poc', 
                   'Average CPI year-on-year inflation rate ': 'ind3-poc', 'Current account balance (as of % GDP) ': 'ind4-poc', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-poc', 'Imports of goods and services (as of % GDP) ': 'ind6-poc',
                  'External debt (as of % GDP) ': 'ind7-poc', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-poc', 'Average exchange rate (HRK : 1 USD) ': 'ind9-poc',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-poc', 'General government debt (as % of GDP) e': 'ind11-poc',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-poc'}, inplace=True)

In [34]:
del test_merged['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']
del test_merged['GODINE']

In [35]:
test_merged.head()

,KLIJENT_ID,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,...,ind3-poc,ind4-poc,ind5-poc,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc
0,1049000,273555.01,2,1410,TM0109,L,3.60,A,64,NaN,...,3.4,-0.114238,41.6252,41.1878,102.929,7.51734,5.85086,-5.2655,69.4867,43.2
1,481302,50970.78,2,1410,TM0109,L,0.20,A,56,NaN,...,-1.1,2.58247,48.8272,45.993,89.2949,7.52938,6.80372,-0.932065,80.4801,44.5847
2,838192,8969.36,2,1410,TM0109,L,0.02,A,52,NaN,...,1.5,2.63013,51.6353,50.5633,75.4149,7.41411,6.27841,-0.5,74.0661,46.871
3,174575,16142.48,2,1410,TM0109,L,2.30,C,76,NaN,...,3.4,-0.114238,41.6252,41.1878,102.929,7.51734,5.85086,-5.2655,69.4867,43.2
4,14884,45315.31,2,1410,TM0109,L,3.30,C,53,NaN,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8686,44.8


<h3> Priprema za treniranje modela </h3>

U ovom dijelu enkodiramo kategoričke značajke te normaliziramo skupove.

In [36]:
X_train = train_merged.drop(['PRIJEVREMENI_RASKID'], axis = 1)
X_test = test_merged.drop(['PRIJEVREMENI_RASKID'], axis = 1)

y_train = train_merged['PRIJEVREMENI_RASKID']
y_test = test_merged['PRIJEVREMENI_RASKID'] # to je inace samo nan

X_train['train'] = 1
X_test['train'] = 0

<h4> Enkodiranje </h4>

In [37]:
combined = pd.concat([X_train,X_test])

In [38]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

dumm = pd.get_dummies(combined['VRSTA_KLIJENTA'])
combined = pd.concat([combined,dumm],axis = 1)
combined['VRSTA_KLIJENTA_CODE'] = lb_make.fit_transform(combined['VRSTA_KLIJENTA'])
del combined['VRSTA_KLIJENTA']
combined.drop(dumm.columns, axis = 1, inplace = True)


dumm = pd.get_dummies(combined['VALUTA'])
combined = pd.concat([combined,dumm],axis = 1)
combined['VALUTA_CODE'] = lb_make.fit_transform(combined['VALUTA'])
del combined['VALUTA']
combined.drop(dumm.columns, axis = 1, inplace = True)


dumm = pd.get_dummies(combined['TIP_KAMATE'])
combined = pd.concat([combined,dumm],axis = 1)
combined['TIP_KAMATE_CODE'] = lb_make.fit_transform(combined['TIP_KAMATE'])
del combined['TIP_KAMATE']
combined.drop(dumm.columns, axis = 1, inplace = True)

dumm = pd.get_dummies(combined['PROIZVOD'])
combined = pd.concat([combined,dumm],axis = 1)
combined['PROIZVOD_CODE'] = lb_make.fit_transform(combined['PROIZVOD'])
del combined['PROIZVOD']
combined.drop(dumm.columns, axis = 1, inplace = True)

dumm = pd.get_dummies(combined['GODINA_OTVARANJA'])
combined = pd.concat([combined,dumm],axis = 1)
combined['GODINA_OTVARANJA_CODE'] = lb_make.fit_transform(combined['GODINA_OTVARANJA'])
del combined['GODINA_OTVARANJA']
combined.drop(dumm.columns, axis = 1, inplace = True)

dumm = pd.get_dummies(combined['VRSTA_PROIZVODA'])
combined = pd.concat([combined,dumm],axis = 1)
combined['VRSTA_PROIZVODA_CODE'] = lb_make.fit_transform(combined['VRSTA_PROIZVODA'])
del combined['VRSTA_PROIZVODA']
combined.drop(dumm.columns, axis = 1, inplace = True)

In [39]:
### ZAMJENA S NaN I KONVERZIJA TIPOVA

combined['ind1-poc'] = combined['ind1-poc'].astype(float)
combined['ind2-poc'] = combined['ind2-poc'].astype(float)
combined['ind3-poc'] = combined['ind3-poc'].astype(float)
combined.loc[combined['ind4-poc'] == '-','ind4-poc'] = np.nan
combined.loc[combined['ind5-poc'] == '-','ind5-poc'] = np.nan
combined.loc[combined['ind6-poc'] == '-','ind6-poc'] = np.nan
combined.loc[combined['ind10-poc'] == '-','ind10-poc'] = np.nan
combined.loc[combined['ind11-poc'] == '-','ind11-poc'] = np.nan
combined.loc[combined['ind12-poc'] == '-','ind12-poc'] = np.nan
combined['ind4-poc'] = combined['ind4-poc'].astype(float)
combined['ind5-poc'] = combined['ind5-poc'].astype(float)
combined['ind6-poc'] = combined['ind6-poc'].astype(float)
#combined['ind10-poc'] = combined['ind10-poc'].astype(float)
combined['ind11-poc'] = combined['ind11-poc'].astype(float)
combined['ind12-poc'] = combined['ind12-poc'].astype(float)
combined.loc[combined['ind10-poc'] == '....','ind10-poc'] = np.nan
combined['ind10-poc'] = combined['ind10-poc'].astype(float)
combined['ind7-poc'] = combined['ind7-poc'].astype(float)
combined['ind8-poc'] = combined['ind8-poc'].astype(float)
combined['ind9-poc'] = combined['ind9-poc'].astype(float)

In [40]:
y_train = lb_make.fit_transform(y_train)
X_train = combined[combined['train']==1]
X_test = combined[combined['train']==0]
X_train.drop(['train'],axis=1,inplace=True)
X_test.drop(['train'],axis=1,inplace=True)

In [41]:
X_train.shape

(663500, 24)

In [42]:
X_test.shape

(84903, 24)

In [43]:
X_train.isnull().sum()

KLIJENT_ID                      0
PLANIRANA_DULJINA_UGOVORA       0
PRODULJENJA                     0
STAROST                         0
UGOVORENI_IZNOS                 0
VISINA_KAMATE                   0
ind1-poc                       60
ind10-poc                    2368
ind11-poc                      60
ind12-poc                      60
ind2-poc                       60
ind3-poc                       60
ind4-poc                       60
ind5-poc                       60
ind6-poc                       60
ind7-poc                       60
ind8-poc                       60
ind9-poc                       60
VRSTA_KLIJENTA_CODE             0
VALUTA_CODE                     0
TIP_KAMATE_CODE                 0
PROIZVOD_CODE                   0
GODINA_OTVARANJA_CODE           0
VRSTA_PROIZVODA_CODE            0
dtype: int64

In [44]:
X_test.isnull().sum()

KLIJENT_ID                      0
PLANIRANA_DULJINA_UGOVORA       0
PRODULJENJA                     0
STAROST                         0
UGOVORENI_IZNOS                 0
VISINA_KAMATE                1290
ind1-poc                       29
ind10-poc                     709
ind11-poc                      29
ind12-poc                      29
ind2-poc                       29
ind3-poc                       29
ind4-poc                       29
ind5-poc                       29
ind6-poc                       29
ind7-poc                       29
ind8-poc                       29
ind9-poc                       29
VRSTA_KLIJENTA_CODE             0
VALUTA_CODE                     0
TIP_KAMATE_CODE                 0
PROIZVOD_CODE                   0
GODINA_OTVARANJA_CODE           0
VRSTA_PROIZVODA_CODE            0
dtype: int64

<h4> Popunjavanje nedostajućih vrijednosti </h4>

Zbog toga što u tablici makroekonomskih podataka nedostaju vrijednosti za godine prije 2000., a postoje ugovori koji su tada otvoreni, pojavile su se nedostajuće vrijednosti u oba skupa. 

In [45]:
from sklearn.preprocessing import Imputer
# Create an imputer object that looks for 'Nan' values, then replaces them with the mean value of the feature by columns (axis=0)
mean_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

# Train the imputor on the df dataset
imputed_df = pd.DataFrame(mean_imputer.fit_transform(X_train))
imputed_df.columns = X_train.columns
imputed_df.index = X_train.index

X_train = imputed_df

# Train the imputor on the test dataset
imputed_df1 = pd.DataFrame(mean_imputer.fit_transform(X_test))
imputed_df1.columns = X_test.columns
imputed_df1.index = X_test.index

X_test=imputed_df1

C:\Users\dorot\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


<h4> Normalizacija </h4>

Za skaliranje podataka koristimo MinMaxScaler.

In [46]:
from sklearn import preprocessing

x_train = X_train.values 
x_test = X_test.values
min_max_scaler = preprocessing.MinMaxScaler()
x_train_scaled = min_max_scaler.fit_transform(x_train)
x_test_scaled = min_max_scaler.transform(x_test)

new_X_train = pd.DataFrame(x_train_scaled)
new_X_test = pd.DataFrame(x_test_scaled)

new_X_train.columns = X_train.columns
new_X_train.index = X_train.index

new_X_test.columns = X_test.columns
new_X_test.index = X_test.index

X_train = new_X_train
X_test = new_X_test

In [47]:
X_train['PRIJEVREMENI_RASKID'] = y_train

In [48]:
X_train.to_hdf('X_train.h5', key = 'X_train')

In [49]:
X_test.to_hdf('X_test.h5', key = 'X_test')